In [1]:
# A dictionary of movie critics and their ratings of a small
# set of movies
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [2]:
from math import sqrt
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
                      for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

In [3]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

In [4]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1
    # Find the number of elements
    n=len(si)
    # if they are no ratings in common, return 0
    if n==0: return 0
    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r

In [5]:
sim_pearson(critics,'Lisa Rose','Gene Seymour')

0.39605901719066977

In [6]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result

In [7]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    # Sort the list so the highest scores appear at the top 
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [8]:
topMatches(critics,'Toby', n = 3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

In [9]:
def calculateSimilarItems(prefs,n=10):
    # Create a dictionary of items showing which other items they
    # are most similar to.
    result={}
    # Invert the preference matrix to be item-centric
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        # Status updates for large datasets
        c+=1
        if c%100==0: print("%d / %d" % (c,len(itemPrefs)))
        # Find the most similar items to this one
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [10]:
itemsim=calculateSimilarItems(critics)
itemsim

{'Just My Luck': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'You, Me and Dupree'),
  (0.15384615384615385, 'The Night Listener'),
  (0.10526315789473684, 'Snakes on a Plane'),
  (0.06451612903225806, 'Superman Returns')],
 'Lady in the Water': [(0.4, 'You, Me and Dupree'),
  (0.2857142857142857, 'The Night Listener'),
  (0.2222222222222222, 'Snakes on a Plane'),
  (0.2222222222222222, 'Just My Luck'),
  (0.09090909090909091, 'Superman Returns')],
 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'The Night Listener'),
  (0.16666666666666666, 'Superman Returns'),
  (0.10526315789473684, 'Just My Luck'),
  (0.05128205128205128, 'You, Me and Dupree')],
 'Superman Returns': [(0.16666666666666666, 'Snakes on a Plane'),
  (0.10256410256410256, 'The Night Listener'),
  (0.09090909090909091, 'Lady in the Water'),
  (0.06451612903225806, 'Just My Luck'),
  (0.05333333333333334, 'You, Me and Dupree')],
 'The Night Listener': [(0.28

In [11]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
    # don't compare me to myself
        if other==person: continue
        sim=similarity(prefs,person,other)
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
            # Similarity * Score 
                totals.setdefault(item,0) 
                totals[item]+=prefs[other][item]*sim 
                # Sum of similarities 
                simSums.setdefault(item,0) 
                simSums[item]+=sim
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items()]
    # Return the sorted list 
    rankings.sort( ) 
    rankings.reverse( ) 
    return rankings

In [12]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}
    # Loop over items rated by this user
    for (item,rating) in userRatings.items():
        # Loop over items similar to this one
        for (similarity,item2) in itemMatch[item]:
            # Ignore if this user has already rated this item
            if item2 in userRatings: continue
            # Weighted sum of rating times similarity
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
            # Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
    # Divide each total score by total weighting to get an average 
    rankings=[(score/totalSim[item],item) for item,score in scores.items()]
    # Return the rankings from highest to lowest 
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [13]:
getRecommendedItems(critics,itemsim,'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

In [23]:
import io
def loadMovieLens(path='resources\ml_100k'):
    # Get movie titles
    movies={}
    for line in io.open(path+'/u.item', encoding = "ISO-8859-1"):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [29]:
prefs=loadMovieLens()
prefs['87']

{u'2001: A Space Odyssey (1968)': 5.0,
 u'Ace Ventura: Pet Detective (1994)': 4.0,
 u'Addams Family Values (1993)': 2.0,
 u'Addicted to Love (1997)': 4.0,
 u'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
 u'Adventures of Robin Hood, The (1938)': 5.0,
 u'Air Force One (1997)': 3.0,
 u'Air Up There, The (1994)': 3.0,
 u'Alien (1979)': 4.0,
 u'American President, The (1995)': 5.0,
 u'Annie Hall (1977)': 4.0,
 u'Apocalypse Now (1979)': 4.0,
 u'Babe (1995)': 5.0,
 u'Baby-Sitters Club, The (1995)': 2.0,
 u'Back to the Future (1985)': 5.0,
 u'Bad Boys (1995)': 4.0,
 u'Bananas (1971)': 5.0,
 u'Barcelona (1994)': 3.0,
 u'Batman & Robin (1997)': 4.0,
 u'Batman (1989)': 3.0,
 u'Batman Returns (1992)': 3.0,
 u'Big Green, The (1995)': 3.0,
 u'Big Squeeze, The (1996)': 2.0,
 u'Birdcage, The (1996)': 4.0,
 u'Blade Runner (1982)': 4.0,
 u'Blues Brothers, The (1980)': 5.0,
 u'Boomerang (1992)': 3.0,
 u'Boot, Das (1981)': 4.0,
 u'Brady Bunch Movie, The (1995)': 2.0,
 u'Braveheart (1995

In [30]:
getRecommendations(prefs,'87')[0:30]

[(5.0, u'They Made Me a Criminal (1939)'),
 (5.0, u'Star Kid (1997)'),
 (5.0, u'Santa with Muscles (1996)'),
 (5.0, u'Saint of Fort Washington, The (1993)'),
 (5.0, u'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, u'Great Day in Harlem, A (1994)'),
 (5.0, u'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, u'Boys, Les (1997)'),
 (4.89884443128923, u'Legal Deceit (1997)'),
 (4.815019082242709, u'Letter From Death Row, A (1998)'),
 (4.7321082983941425, u'Hearts and Minds (1996)'),
 (4.696244466490867, u'Pather Panchali (1955)'),
 (4.652397061026758, u'Lamerica (1994)'),
 (4.538723693474813, u'Leading Man, The (1996)'),
 (4.535081339106103, u'Mrs. Dalloway (1997)'),
 (4.532337612572981, u'Innocents, The (1961)'),
 (4.527998574747079, u'Casablanca (1942)'),
 (4.510270149719864, u'Everest (1998)'),
 (4.493967755428439, u'Dangerous Beauty (1998)'),
 (4.485151301801342,
  u'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.463287461290222, u'Wrong Trousers, The (19

In [31]:
itemsim=calculateSimilarItems(prefs,n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [32]:
getRecommendedItems(prefs,itemsim,'87')[0:30]

[(5.0, u'U Turn (1997)'),
 (5.0, u'Star Maps (1997)'),
 (5.0, u'Shiloh (1997)'),
 (5.0, u'Palmetto (1998)'),
 (5.0, u'Newton Boys, The (1998)'),
 (5.0, u'Nelly & Monsieur Arnaud (1995)'),
 (5.0, u'Mr. Wonderful (1993)'),
 (5.0, u'Mina Tannenbaum (1994)'),
 (5.0, u'Manny & Lo (1996)'),
 (5.0, u'Love Is All There Is (1996)'),
 (5.0, u'Losing Chase (1996)'),
 (5.0, u'Letter From Death Row, A (1998)'),
 (5.0, u'Late Bloomers (1996)'),
 (5.0, u'Last Klezmer: Leopold Kozlowski, His Life and Music, The (1995)'),
 (5.0, u'Johns (1996)'),
 (5.0, u'Innocent Sleep, The (1995)'),
 (5.0, u'He Walked by Night (1948)'),
 (5.0, u'Further Gesture, A (1996)'),
 (5.0, u'For the Moment (1994)'),
 (5.0, u'Fear, The (1995)'),
 (5.0, u'Every Other Weekend (1990)'),
 (5.0, u'Everest (1998)'),
 (5.0, u'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, u'Damsel in Distress, A (1937)'),
 (5.0, u'Cyclo (1995)'),
 (4.833333333333333, u'Marlene Dietrich: Shadow and Light (1996) '),
 (4.8, u'Mamma Roma (19